In [5]:
# Cell 1: Import Required Libraries
# =====================================

# Core Python libraries
import json
import os
from typing import Dict, List, Any, Optional, Tuple
from datetime import datetime
import base64
from io import BytesIO

# Data manipulation and analysis
import pandas as pd
import numpy as np

# Visualization libraries
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import seaborn as sns

# HTTP requests for API calls
import requests

# Environment variables (for API keys)
from dotenv import load_dotenv

# Rich text formatting for better notebook output
from IPython.display import display, HTML, Image, Markdown

# Warnings management
import warnings
warnings.filterwarnings('ignore')

# Set visualization defaults
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Load environment variables if .env file exists
load_dotenv()

False

In [8]:
# Cell 2: Configuration and API Setup
# =====================================

# API Configuration
OPENROUTER_API_KEY = "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"
OPENROUTER_BASE_URL = "https://openrouter.ai/api/v1/chat/completions"
MODEL_NAME = "deepseek/deepseek-chat-v3-0324:free"  # Deepseek model via OpenRouter

# Headers for API requests
HEADERS = {
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
    "Content-Type": "application/json",
    "HTTP-Referer": "http://localhost:8888",  # Required for OpenRouter
    "X-Title": "Dental Practice Valuation Q&A"  # Optional, helps with OpenRouter analytics
}

# Question Categories (for routing to appropriate handlers)
QUESTION_CATEGORIES = {
    "P&L_ANALYSIS": ["profit", "revenue", "expense", "margin", "ebitda", "income"],
    "TAX": ["tax", "deduction", "write-off", "depreciation", "amortization"],
    "VALUATION": ["multiple", "valuation", "worth", "value", "price"],
    "OPERATIONS": ["patient", "operatories", "technology", "location", "service"],
    "SDE": ["sde", "seller", "discretionary", "earnings", "addback", "adjustment"],
    "SWOT": ["strength", "weakness", "opportunity", "threat", "swot", "growth"],
    "DEBT": ["debt", "loan", "financing", "interest", "service"],
    "GENERAL": ["tell me about", "overview", "summary", "explain"]
}

# Visualization preferences
VIZ_CONFIG = {
    "figure_size": (10, 6),
    "color_scheme": ["#2E86AB", "#A23B72", "#F18F01", "#C73E1D", "#6A994E"],
    "font_size": 12,
    "title_size": 14,
    "dpi": 100,
    "style": "seaborn-v0_8-darkgrid"
}

# Financial formatting preferences
FINANCIAL_CONFIG = {
    "currency_symbol": "$",
    "thousands_separator": ",",
    "decimal_places": 0,
    "percentage_decimal": 1
}

# System prompt for the LLM
SYSTEM_PROMPT = """You are a dental practice valuation expert AI assistant. You analyze financial data, 
operational metrics, and practice characteristics to provide insights about dental practice valuations.

Your responses should be:
1. Accurate and based solely on the provided data
2. Professional but accessible to non-financial professionals
3. Focused on actionable insights
4. Clear about calculations and methodologies used

When analyzing data:
- Always show key calculations
- Explain financial terms when first used
- Highlight important ratios and benchmarks
- Provide context for valuation multiples
"""

# Test API connection
def test_api_connection():
    """Test the OpenRouter API connection with Deepseek"""
    test_payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "Respond with 'Connection successful!' if you receive this."}
        ],
        "max_tokens": 50,
        "temperature": 0.1
    }
    
    try:
        response = requests.post(
            OPENROUTER_BASE_URL,
            headers=HEADERS,
            json=test_payload
        )
        
        if response.status_code == 200:
            result = response.json()
            message = result['choices'][0]['message']['content']
            print(f"✅ API Connection Test Successful!")
            print(f"📡 Model: {MODEL_NAME}")
            print(f"💬 Response: {message}")
            return True
        else:
            print(f"❌ API Connection Failed!")
            print(f"Status Code: {response.status_code}")
            print(f"Error: {response.text}")
            return False
            
    except Exception as e:
        print(f"❌ Connection Error: {str(e)}")
        return False

# Run connection test
print("=" * 50)
print("DENTAL PRACTICE VALUATION Q&A SYSTEM")
print("=" * 50)
print(f"\n🔧 Configuration loaded successfully!")
print(f"🤖 Using model: {MODEL_NAME}")
print(f"\n📡 Testing API connection...")
print("-" * 50)

connection_status = test_api_connection()

if connection_status:
    print("-" * 50)
    print("✅ System ready for use!")
else:
    print("-" * 50)
    print("⚠️  Please check your API key and connection")

DENTAL PRACTICE VALUATION Q&A SYSTEM

🔧 Configuration loaded successfully!
🤖 Using model: deepseek/deepseek-chat-v3-0324:free

📡 Testing API connection...
--------------------------------------------------
✅ API Connection Test Successful!
📡 Model: deepseek/deepseek-chat-v3-0324:free
💬 Response: Connection successful!
--------------------------------------------------
✅ System ready for use!


In [10]:
# Cell 3: Data Loading and Validation Functions
# ===============================================

class DentalPracticeData:
    """Class to handle dental practice valuation data"""
    
    def __init__(self, json_path: str = None, json_data: dict = None):
        """Initialize with either a file path or direct JSON data"""
        if json_path:
            self.data = self.load_from_file(json_path)
        elif json_data:
            self.data = json_data
        else:
            raise ValueError("Must provide either json_path or json_data")
        
        self.validate_data()
        self.calculate_derived_metrics()
    
    def load_from_file(self, filepath: str) -> dict:
        """Load JSON data from file"""
        try:
            with open(filepath, 'r') as f:
                data = json.load(f)
            print(f"✅ Successfully loaded data from {filepath}")
            return data
        except FileNotFoundError:
            print(f"❌ File not found: {filepath}")
            raise
        except json.JSONDecodeError as e:
            print(f"❌ Invalid JSON format: {e}")
            raise
    
    def validate_data(self):
        """Validate that all required fields are present"""
        required_fields = [
            "Clinic", "Revenue_Trailing12", "EBITDA_Trailing12", 
            "Total_SDE", "SDE", "Valuation_Multiples"
        ]
        
        missing_fields = []
        for field in required_fields:
            if field not in self.data:
                missing_fields.append(field)
        
        if missing_fields:
            raise ValueError(f"Missing required fields: {missing_fields}")
        
        # Validate SDE components
        sde_components = ["Net_Profit_pre_tax", "Owner_Salary_minus_Market", 
                         "Owner_Perks", "Interest", "Depreciation", 
                         "Amortization", "Non_Recurring"]
        
        if "SDE" in self.data:
            missing_sde = [c for c in sde_components if c not in self.data["SDE"]]
            if missing_sde:
                print(f"⚠️  Warning: Missing SDE components: {missing_sde}")
        
        print("✅ Data validation successful!")
    
    def calculate_derived_metrics(self):
        """Calculate additional metrics from the base data"""
        # Calculate key ratios
        self.metrics = {}
        
        # Profitability metrics
        revenue = self.data.get("Revenue_Trailing12", 0)
        if revenue > 0:
            self.metrics["EBITDA_Margin"] = (self.data.get("EBITDA_Trailing12", 0) / revenue) * 100
            self.metrics["SDE_Margin"] = (self.data.get("Total_SDE", 0) / revenue) * 100
        
        # Valuation metrics
        if "Valuation_Multiples" in self.data:
            multiples = self.data["Valuation_Multiples"]
            self.metrics["Implied_Value_EBITDA"] = (
                self.data.get("EBITDA_Trailing12", 0) * 
                multiples.get("EBITDA_multiple", 0)
            )
            self.metrics["Implied_Value_Revenue"] = (
                revenue * multiples.get("Revenue_multiple", 0)
            )
        
        # Debt service coverage
        if "Debt_Service_Annual" in self.data and self.data["Debt_Service_Annual"] > 0:
            self.metrics["DSCR"] = (
                self.data.get("EBITDA_Trailing12", 0) / 
                self.data["Debt_Service_Annual"]
            )
        
        # After-tax cash flow (if buyer tax rate provided)
        if "Buyer_Tax_Rate" in self.data:
            tax_rate = self.data["Buyer_Tax_Rate"]
            self.metrics["After_Tax_SDE"] = (
                self.data.get("Total_SDE", 0) * (1 - tax_rate)
            )
            
            if "Debt_Service_Annual" in self.data:
                self.metrics["After_Tax_Cash_Flow"] = (
                    self.metrics["After_Tax_SDE"] - 
                    self.data["Debt_Service_Annual"]
                )
    
    def get_summary(self) -> pd.DataFrame:
        """Get a summary DataFrame of key metrics"""
        summary_data = {
            "Metric": [],
            "Value": [],
            "Category": []
        }
        
        # Revenue metrics
        summary_data["Metric"].append("Revenue (TTM)")
        summary_data["Value"].append(f"${self.data.get('Revenue_Trailing12', 0):,.0f}")
        summary_data["Category"].append("Revenue")
        
        # Profitability metrics
        summary_data["Metric"].append("EBITDA (TTM)")
        summary_data["Value"].append(f"${self.data.get('EBITDA_Trailing12', 0):,.0f}")
        summary_data["Category"].append("Profitability")
        
        summary_data["Metric"].append("Total SDE")
        summary_data["Value"].append(f"${self.data.get('Total_SDE', 0):,.0f}")
        summary_data["Category"].append("Profitability")
        
        # Margin metrics
        if "EBITDA_Margin" in self.metrics:
            summary_data["Metric"].append("EBITDA Margin")
            summary_data["Value"].append(f"{self.metrics['EBITDA_Margin']:.1f}%")
            summary_data["Category"].append("Profitability")
        
        if "SDE_Margin" in self.metrics:
            summary_data["Metric"].append("SDE Margin")
            summary_data["Value"].append(f"{self.metrics['SDE_Margin']:.1f}%")
            summary_data["Category"].append("Profitability")
        
        # Valuation metrics
        if "Implied_Value_EBITDA" in self.metrics:
            summary_data["Metric"].append("Implied Value (EBITDA)")
            summary_data["Value"].append(f"${self.metrics['Implied_Value_EBITDA']:,.0f}")
            summary_data["Category"].append("Valuation")
        
        if "Implied_Value_Revenue" in self.metrics:
            summary_data["Metric"].append("Implied Value (Revenue)")
            summary_data["Value"].append(f"${self.metrics['Implied_Value_Revenue']:,.0f}")
            summary_data["Category"].append("Valuation")
        
        # Debt metrics
        if "DSCR" in self.metrics:
            summary_data["Metric"].append("Debt Service Coverage")
            summary_data["Value"].append(f"{self.metrics['DSCR']:.2f}x")
            summary_data["Category"].append("Debt")
        
        return pd.DataFrame(summary_data)
    
    def display_practice_overview(self):
        """Display a formatted overview of the practice"""
        print("\n" + "="*60)
        print(f"PRACTICE: {self.data.get('Clinic', 'Unknown')}")
        print("="*60)
        
        if "Practice_Overview" in self.data:
            overview = self.data["Practice_Overview"]
            print("\n📍 PRACTICE DETAILS:")
            print(f"   Location: {overview.get('Location', 'N/A')}")
            print(f"   Patient Base: {overview.get('Patient_Base', 'N/A')}")
            print(f"   Operatories: {overview.get('Operatories', 'N/A')}")
            if "Technology" in overview:
                print(f"   Technology: {', '.join(overview['Technology'])}")
        
        print("\n💰 KEY FINANCIAL METRICS:")
        summary_df = self.get_summary()
        for _, row in summary_df.iterrows():
            print(f"   {row['Metric']}: {row['Value']}")
        
        if "SWOT" in self.data:
            swot = self.data["SWOT"]
            print("\n📊 SWOT HIGHLIGHTS:")
            if "Strengths" in swot and swot["Strengths"]:
                print(f"   Key Strength: {swot['Strengths'][0]}")
            if "Opportunities" in swot and swot["Opportunities"]:
                print(f"   Top Opportunity: {swot['Opportunities'][0]}")

# Test with the demo data
print("Testing data loading with demo_clinic.json...")
print("-" * 60)

# Load the demo data
try:
    practice_data = DentalPracticeData(json_path="demo_clinic.json")
    practice_data.display_practice_overview()
    
    print("\n📊 Summary DataFrame:")
    display(practice_data.get_summary())
    
    print("\n✅ Data loading module ready!")
    
except FileNotFoundError:
    print("\n⚠️  demo_clinic.json not found. Creating from provided data...")
    
    # Create the demo file from the data you provided
    demo_data = {
        "Clinic": "Demo Dental NYC",
        "Valuation_Date": "2025-07-22",
        "Revenue_Trailing12": 1546696,
        "EBITDA_Trailing12": 545301,
        "Total_SDE": 460000,
        "SDE": {
            "Net_Profit_pre_tax": 350000,
            "Owner_Salary_minus_Market": 60000,
            "Owner_Perks": 18000,
            "Interest": 12000,
            "Depreciation": 9000,
            "Amortization": 4000,
            "Non_Recurring": 7000
        },
        "Addback_Justifications": {
            "Net_Profit_pre_tax": "Base accounting profit before income tax adjustments.",
            "Owner_Salary_minus_Market": "Portion of DDS salary above fair‑market compensation.",
            "Owner_Perks": "Clinic‑paid personal auto, phone, and family health insurance.",
            "Interest": "Financing cost—assumed refinanced by buyer, thus added back.",
            "Depreciation": "Non‑cash charge; added back to reflect cash flow.",
            "Amortization": "Non‑cash; pertains to prior goodwill amortization.",
            "Non_Recurring": "One‑time legal settlement expense in FY 2024."
        },
        "Valuation_Multiples": {"EBITDA_multiple": 4.5, "Revenue_multiple": 0.8},
        "Debt_Service_Annual": 168008,
        "Buyer_Tax_Rate": 0.25,
        "Practice_Overview": {
            "Location": "Midtown Manhattan, NY",
            "Patient_Base": "95% Fee‑For‑Service, 2100 active patients",
            "Operatories": 6,
            "Technology": ["Digital X‑ray", "Itero Scanner", "CBCT"]
        },
        "SWOT": {
            "Strengths": ["High FFS revenue", "Prime Manhattan location", "Robust hygiene program"],
            "Weaknesses": ["Limited Saturday hours"],
            "Opportunities": ["Add implant services", "Expand evening hours"],
            "Threats": ["Rising rent costs"]
        }
    }
    
    # Save to file
    with open("demo_clinic.json", "w") as f:
        json.dump(demo_data, f, indent=2)
    
    print("✅ Created demo_clinic.json")
    
    # Now load it
    practice_data = DentalPracticeData(json_path="demo_clinic.json")
    practice_data.display_practice_overview()
    
    print("\n📊 Summary DataFrame:")
    display(practice_data.get_summary())
    
    print("\n✅ Data loading module ready!")

Testing data loading with demo_clinic.json...
------------------------------------------------------------
✅ Successfully loaded data from demo_clinic.json
✅ Data validation successful!

PRACTICE: Demo Dental NYC

📍 PRACTICE DETAILS:
   Location: Midtown Manhattan, NY
   Patient Base: 95% Fee‑For‑Service, 2100 active patients
   Operatories: 6
   Technology: Digital X‑ray, Itero Scanner, CBCT

💰 KEY FINANCIAL METRICS:
   Revenue (TTM): $1,546,696
   EBITDA (TTM): $545,301
   Total SDE: $460,000
   EBITDA Margin: 35.3%
   SDE Margin: 29.7%
   Implied Value (EBITDA): $2,453,854
   Implied Value (Revenue): $1,237,357
   Debt Service Coverage: 3.25x

📊 SWOT HIGHLIGHTS:
   Key Strength: High FFS revenue
   Top Opportunity: Add implant services

📊 Summary DataFrame:


,Metric,Value,Category
0,Revenue (TTM),"$1,546,696",Revenue
1,EBITDA (TTM),"$545,301",Profitability
2,Total SDE,"$460,000",Profitability
3,EBITDA Margin,35.3%,Profitability
4,SDE Margin,29.7%,Profitability
5,Implied Value (EBITDA),"$2,453,854",Valuation
6,Implied Value (Revenue),"$1,237,357",Valuation
7,Debt Service Coverage,3.25x,Debt



✅ Data loading module ready!
